In [ ]:
# what model am i selecting
# why am i selecting the model
# evaluate the results
# use test data
# possibly try another model
# end with conclusion

# Introduction:
In this notebook, I will present my solution and analysis of the Titanic dataset.  This is a very famous dataset that can be found on [kaggle](). The objective is to build a model to correctly classify who will survive in from the titanic. . Throughout this notebook, I will meticulously explore the data, explain what im doing, construct and evaluate models, and critically assess the results.

# Project Overview:
The primary objective of this project is to delve into the Titanic dataset, which contains information about passengers, including their demographics and survival status. By carefully studying the data, I will identify meaningful patterns and relationships that could help me in predicting survival outcomes. To achieve this, I will follow a structured approach: 

**Data Exploration:** I will conduct a comprehensive exploration of the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables.

**Data Preprocessing:** Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure optimal model performance.

**Model Building:** I will experiment with several machine learning algorithms, such as logistic regression, decision trees, random forests, and support vector machines, to construct predictive models based on the preprocessed data.

**Model Evaluation:** To gauge the performance of the models, I will utilize appropriate evaluation metrics and techniques such as cross-validation to ensure robustness and mitigate overfitting.

**Result Analysis:** Finally, I will interpret the results of the models, identifying significant factors that contribute to passenger survival prediction and discussing potential areas for model improvement.

**Conclusion:**
The Titanic dataset is a good challange to test your knowledege on machine learning. This will serve as a good test for me to keep learning and testing my skills.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Exploration
As mentioned earlier I have got the dataset from kaggle. The link to that can be found above. The download came with two csv files. One for the training set and one for the test set. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [ ]:
train = pd.read_csv('./train.csv') # read the data

Now that we have our data we want to see what it contains. First we will get its size. 

In [16]:
train.size # get size of train data

10692

As we can see it is somewhat large. This will be good for our model to have many samples to learn from. Additionally its not too large where we would require lots of compute power and time for training. Moving on from this we can check what our data consists of. 

In [18]:
train.info() # get info on our train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


From the table above us we can gather a few things. First thing is that we have quite a big ammount of missing values. 

In [15]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object